# Basic RNNs

In [ ]:
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf

In [ ]:
X0=tf.placeholder(tf.float32,[None, n_inputs])

In [ ]:
X1=tf.placeholder(tf.float32,[None, n_inputs])

In [ ]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
b= tf.Variable(tf.zeros([1,n_neurons], dtype=tf.float32))

In [ ]:
Y0 = tf.tanh(tf.matmul(X0, Wx) +b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) +b)
#tanh is the activation function. in dynamic rnn, by default in tf.contrib.rnn.BasicRNNCell, it is tanh.
#however, you can change it to any other activation function using the argument "activation".

In [ ]:
init=tf.global_variables_initializer()

In [ ]:
import numpy as np

In [ ]:
X0_batch =  np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])

In [ ]:
import pandas as pd
X = pd.DataFrame(X0_batch)

In [ ]:
X

In [ ]:
X1_batch =  np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

In [ ]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [ ]:
print(Y0_val)

In [ ]:
print(Y1_val)

# Static Unrolling Through Time

In [ ]:
import tensorflow as tf

In [ ]:
n_inputs = 3
n_neurons = 5

In [ ]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1], dtype=tf.float32)
Y0, Y1=output_seqs
#function returns two objects. first is a python list containing the output tensors for each time step. 
#The second is a tensor containing the final states of the network. When you are using basic cells, the final state is simply
#equal to last output.

In [ ]:
import numpy as np
X0_batch =  np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch =  np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    Y0_val, Y1_val = sess.run([Y0,Y1],feed_dict={X0: X0_batch, X1:X1_batch })

In [ ]:
print(Y0_val)

In [ ]:
print(Y1_val)

If there were 50 time steps, it would not be very convenient to have to define 50 input placeholders and 50 output tensors. We can simplify this.

it takes a single input placeholder of shape  [None, n_steps, n_inputs] where the first dimension isthe mini-batch size.

In [ ]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf
X=tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,X_seqs, dtype=tf.float32)

In [ ]:
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [ ]:
import numpy as np
X_batch = np.array([ 
    [[0,1,2],[9,8,7]],#instance 0
    [[3,4,5],[0,0,0]],#instance 1
    [[6,7,8],[6,5,4]],#instance 2
    [[9,0,1],[3,2,1]]])#instance 3

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X:X_batch})

In [ ]:
print(outputs_val)

In [ ]:
print(states_val)

# Dynamic Unrolling Through Time

In [ ]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf
X=tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [ ]:
import numpy as np
X_batch = np.array([ 
    [[0,1,2],[9,8,7]],#instance 0
    [[3,4,5],[0,0,0]],#instance 1
    [[6,7,8],[6,5,4]],#instance 2
    [[9,0,1],[3,2,1]]])#instance 3

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X:X_batch})

In [ ]:
print(outputs_val)
print(outputs_val.shape)

print(states_val)
print(states_val.shape)

## BASIC LSTM EXAMPLE

In [1]:
import tensorflow as tf
import numpy as np
X_batch = np.array([ 
    [[0,1,2],[9,8,7]],#instance 0
    [[3,4,5],[0,0,0]],#instance 1
    [[6,7,8],[6,5,4]],#instance 2
    [[9,0,1],[3,2,1]]])#instance 3

n_steps = 2
n_inputs = 3
n_neurons = 5

X=tf.placeholder(tf.float32, [None, n_steps, n_inputs])

# create 2 LSTMCells
rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [128, 256]]
#First cell has number of units as 128 and the second cell has 256

# create a RNN cell composed sequentially of a number of RNNCells
multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

# 'outputs' is a tensor of shape [batch_size, max_time, 256]
# 'state' is a N-tuple where N is the number of LSTMCells containing a
# tf.contrib.rnn.LSTMStateTuple for each cell
outputs, states = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=X,
                                   dtype=tf.float32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X:X_batch})
    

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
print(outputs_val)
print(outputs_val.shape) #(4, 2, 256)
print(states_val) #tuple
#outputs_val and states_val[1].h will be the same

[[[ 0.00769336 -0.00032055 -0.0019508  ...  0.00164839 -0.00298414
   -0.00504108]
  [ 0.06101465 -0.00499493 -0.00471805 ...  0.01425603 -0.02972551
   -0.02672425]]

 [[ 0.03041571 -0.00122588 -0.00342316 ...  0.00724636 -0.01426015
   -0.01749035]
  [ 0.04229382 -0.00407521 -0.01046767 ...  0.01387198 -0.02256514
   -0.02741067]]

 [[ 0.04720089 -0.00186801 -0.00530894 ...  0.00951193 -0.02356498
   -0.02267964]
  [ 0.09486194 -0.00958257 -0.01041569 ...  0.02924553 -0.05174449
   -0.05033881]]

 [[ 0.02736288 -0.00850442 -0.0024518  ...  0.00669371 -0.00675236
   -0.01343909]
  [ 0.05584113 -0.01584079 -0.00768016 ...  0.01812207 -0.01703749
   -0.03205018]]]
(4, 2, 256)
(LSTMStateTuple(c=array([[ 6.41405284e-01,  2.58917689e-01, -6.24168925e-02,
         3.81408930e-01,  1.17082417e-01, -8.20610523e-02,
        -3.69117349e-01,  6.84079349e-01, -5.06348014e-01,
        -7.35761762e-01, -4.80452292e-02,  4.45271544e-02,
        -2.52230503e-02, -1.03973255e-01, -1.68273553e-01,
   

In [3]:
print(states_val[0].c.shape)
print(states_val[1].c.shape)
print(states_val[0].h.shape)
print(states_val[1].h.shape)

(4, 128)
(4, 256)
(4, 128)
(4, 256)


In [4]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0' shape=(131, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0' shape=(384, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>]

# Handling Variable Length Input Sequences

In [ ]:
import tensorflow as tf
n_neurons = 5
n_inputs = 3
n_steps =2
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length= seq_length)

In [ ]:
import numpy as np
X_batch = np.array([
    [[0,1,2],[9,8,7]], #instance 0 
    [[3,4,5],[0,0,0]], #instance 1
    [[6,7,8],[6,5,4]], #instance 2
    [[9,0,1],[3,2,1]], #instance 3
])

seq_length_batch = np.array([2,1,2,2])

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length:seq_length_batch})

In [ ]:
print(outputs_val)

In [ ]:
print(states_val)
#Second row is only for t=0

# Training a Sequence Classifier

In [ ]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate =0.001

In [ ]:
import tensorflow as tf

In [ ]:
X =  tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int64, [None])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
#dimensionality of states is batch_size X n_neurons

In [ ]:
#tf.layers.dense adds a single layer to your network. The second argument is the number of neurons/nodes of the layer
logits = tf.layers.dense(inputs = states, units = n_outputs)
#Output tensor the same shape as inputs except the last dimension is of size units.

In [ ]:
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)

In [ ]:
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
#correct = tf.equal(tf.argmax(logits, 1), y)
accuracy =tf.reduce_mean(tf.cast(correct,tf.float32))

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("tmp/data/")

In [ ]:
mnist.train.images.reshape((-1, n_steps, n_inputs)).shape

In [ ]:
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
n_epochs = 10
batch_size =50

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for iterations in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training, feed_dict={X: X_batch, y: y_batch})
            states_val = states.eval(feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train Accuracy: ", acc_train, "Test Accuracy: ", acc_test)

In [ ]:
states_val.shape

if you do convert labels into one-hot vector, this is what you should do. 

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn

#import mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

#define constants
#unrolled through 28 time steps
time_steps=28
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=28
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=10
#size of batch
batch_size=128

#Lets now declare placeholders and weights and bias variables which will be used to convert the output of shape 
#[batch_size,num_units] to [batch_size,n_classes] so that correct class can be predicted.

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#Now that we are receiving inputs of shape [batch_size,time_steps,n_input],we need to convert it into a list of 
#tensors of shape [batch_size,n_inputs] of length time_steps so that it can be then fed to static_rnn.

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    #calculating test accuracy
    test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
    test_label = mnist.test.labels[:128]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

# Training to predict Time Series

Let's create the RNN. It will contain 100 recurrent neurons and we will unroll it over 20 time steps since each traiing instance will be 20 inputs long. Each input will contain only one feature (the value at that time). The targets are also sequences of 20 inputs, each containing a single value:

In [ ]:
n_steps = 20
n_neurons =100
n_inputs = 1
n_outputs = 1

In [ ]:
t_min, t_max  = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [ ]:
import numpy as np
t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

In [ ]:
X_batch, y_batch = next_batch(1, n_steps)

In [ ]:
y_batch

In [ ]:
import tensorflow as tf

At each time step we now have an output vector of size 100 (because we have 100 neurons. Yt will be batch_size X n_neurons). But what we actually want is a single output value at each time step. The simplest solution is to wrap the cell in an OutputProjectionWrapper.

In [ ]:
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu), output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [ ]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training = optimizer.minimize(loss)

In [ ]:
saver = tf.train.Saver()
n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training, feed_dict={X:X_batch, y:y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X:X_batch, y:y_batch})
            print(iteration, "\tMSE", mse)
    saver.save(sess, "./my_time_series_model")

In [ ]:
with tf.Session() as sess:                          
    saver.restore(sess, "./my_time_series_model")   
    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

In [ ]:
print(y_pred)

# Without using an OutputProjectionWrapper¶


In [ ]:
import tensorflow as tf 

n_steps = 20
n_inputs = 1
n_neurons = 100

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

n_outputs = 1
learning_rate = 0.001

In [ ]:
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
#in the last line, the outputs is reshaped to [batch_size X n_steps X n_outputs] because in the loss function below,
#we will find squared mean root and y placeholder is shaped as [None, n_steps, n_outputs].

In [ ]:
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:

n_iterationsn_iterat  = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})
    
    saver.save(sess, "./my_time_series_model")

In [ ]:
y_pred

# Deep RNNs

# MultiRNNCell

In [ ]:
n_inputs = 2
n_steps = 5

import tensorflow as tf 
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
n_neurons = 100
n_layers = 3

import numpy as np
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
          for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
X_batch = np.random.rand(2, n_steps, n_inputs)

In [ ]:
X_batch

In [ ]:
with  tf.Session() as sess:
         init.run()
         outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch})

In [ ]:
#outputs_val[0] #first instance (batch =0)
#outputs_val[0][0] - outputs_val[0][4] # first instance and 5 time steps
#outputs_val[0][0][0] - outputs_val[0][0][99] # first instance and first time step and 100 neurons

In [ ]:
outputs_val.shape
#2 instances, 5 times steps, 100 neurons

In [ ]:
#states_val[0] - states_val[2] #number of layers
#states_val[0][0] - states_val[0][1]   #first layer, 2 instances

#states_val[0][0][0] - states_val[0][0][99] #first layer first instance and 100 neurons
#final states of each cell

# Train a LSTM to predict the next word using a sample short story, Aesop’s Fables (DEEP RNN)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [ ]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

If we feed a LSTM with correct sequences from the text of 3 symbols as inputs and 1 labeled symbol, eventually the neural network will learn to predict the next symbol correctly

In [ ]:
# Target log path
logs_path = '/Users/mustafamuratarat'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = '/Users/mustafamuratarat/belling_the_cat.txt'

In [ ]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

In [ ]:
training_data = read_data(training_file)
print("Loaded training data...")

In [ ]:
training_data 

Technically, LSTM inputs can only understand real numbers. A way to convert symbol to number is to assign a unique integer to each symbol based on frequency of occurrence. For example, there are 112 unique symbols in the text above. The function 'build_dataset' builds a dictionary with the following entries [ “,” : 0 ] [ “the” : 1 ], …, [ “council” : 37 ],…,[ “spoke” : 111 ]. The reverse dictionary is also generated since it will be used in decoding the output of LSTM.

In [ ]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [ ]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [ ]:
vocab_size

Similarly, the prediction is a unique integer identifying the index in the reverse dictionary of the predicted symbol. For example, if the prediction is 37, the predicted symbol is actually “council”.

The generation of output may sound simple but actually LSTM produces a 112-element vector (because vocab_size is 112) of probabilities of prediction for the next symbol normalized by the softmax() function. The index of the element with the highest probability is the predicted index of the symbol in the reverse dictionary (ie a one-hot vector). 

In [ ]:
#We are feeding 3 words at each time steps
n_input = 3

# number of units in RNN cell (basically n_neurons)
n_hidden = 512

display_step = 1000

# Parameters
learning_rate = 0.001
training_iters = 50000

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [ ]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
#In the training process, at each step, 3 symbols are retrieved from the training data. 
#These 3 symbols are converted to integers to form the input vector.
        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

#The training label is a one-hot vector coming from the symbol after the 3 input symbols.
        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))

# Applying Dropout

If you build a very deep RNN, it may end up overfitting the training set. In order to prevent that, a common technique is to apply dropout. You can simply add a dropout layer before or after the RNN as usual but if you also want to apply dropout between RNN layers, you need to use a DropoutWrapper.

In [ ]:
t_min, t_max  = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

import numpy as np
t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

In [ ]:
n_inputs = 1
n_neurons = 100
n_layers = 3
n_steps = 20
n_outputs = 1

In [ ]:
import tensorflow as tf
X  = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

Note: the input_keep_prob parameter can be a placeholder, making it possible to set it to any value you want during training, and to 1.0 during testing (effectively turning dropout off).

In [ ]:
keep_prob = tf.placeholder_with_default(1.0, shape=())
cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
         for layer in range(n_layers)]
cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
              for cell in cells]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells_drop)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
learning_rate = 0.01

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

During training, you can feed any value you want to the keep_prob placeholder (typically 0.5)

In [ ]:
n_iterations = 1500
batch_size = 50
train_keep_prob = 0.5

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        _, mse = sess.run([training_op, loss],
                          feed_dict={X: X_batch, y: y_batch,
                                     keep_prob: train_keep_prob})
        if iteration % 100 == 0:                 
            print(iteration, "Training MSE:", mse) 
    
    saver.save(sess, "./my_dropout_time_series_model")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_dropout_time_series_model")

    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

In [ ]:
y_pred #20 time-steps

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

plt.title("Testing the model", fontsize=14)
plt.plot(t_instance[:-1], time_series(t_instance[:-1]), "bo", markersize=10, label="instance")
plt.plot(t_instance[1:], time_series(t_instance[1:]), "w*", markersize=10, label="target")
plt.plot(t_instance[1:], y_pred[0,:,0], "r.", markersize=10, label="prediction")
plt.legend(loc="upper left")
plt.xlabel("Time")

plt.show()

 it seems that Dropout does not help at all in this particular case.

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

x = tf.range(1, 11, dtype=tf.float32)
x = tf.reshape(x, (5, 1, 2))

cell = tf.contrib.rnn.LSTMCell(10)
initial_state = cell.zero_state(5, dtype=tf.float32)

y1, _ = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32, initial_state=initial_state)

y2, _ = tf.nn.dynamic_rnn(
    tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=1.0, output_keep_prob=0.5, state_keep_prob=1.0),
    x,
    dtype=tf.float32,
    initial_state=initial_state)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
y1

<tf.Tensor: id=105, shape=(5, 1, 10), dtype=float32, numpy=
array([[[ 4.36799787e-02,  3.52170318e-02,  1.52844533e-01,
         -4.27856594e-02, -7.67421946e-02,  1.61572874e-01,
          1.14498682e-01, -1.61731184e-01, -1.28882945e-01,
         -3.44755547e-03]],

       [[ 3.67480516e-02,  1.25189677e-01,  2.76421785e-01,
         -1.05942693e-02, -2.89940864e-01,  3.02900434e-01,
          1.16918601e-01, -2.05750301e-01, -1.62743732e-01,
         -8.90007359e-04]],

       [[ 1.57351140e-02,  2.29096889e-01,  3.02731484e-01,
          9.77676478e-04, -4.67667043e-01,  3.81339967e-01,
          8.54122713e-02, -1.89569458e-01, -1.48455247e-01,
         -3.07439404e-05]],

       [[ 5.47854323e-03,  3.31124783e-01,  3.00104082e-01,
          2.45815795e-03, -5.81289172e-01,  4.32718366e-01,
          5.55033647e-02, -1.63088143e-01, -1.20561428e-01,
          1.35644295e-04]],

       [[ 1.78311777e-03,  4.21142966e-01,  2.88616091e-01,
          1.63363572e-03, -6.50413692e-01,  

In [3]:
y2

<tf.Tensor: id=179, shape=(5, 1, 10), dtype=float32, numpy=
array([[[ 0.08735996,  0.        ,  0.        , -0.        ,
         -0.15348439,  0.32314575,  0.        , -0.        ,
         -0.2577659 , -0.        ]],

       [[ 0.0734961 ,  0.        ,  0.        , -0.        ,
         -0.5798817 ,  0.        ,  0.2338372 , -0.        ,
         -0.32548746, -0.        ]],

       [[ 0.03147023,  0.45819378,  0.60546297,  0.        ,
         -0.        ,  0.76267993,  0.        , -0.        ,
         -0.2969105 , -0.        ]],

       [[ 0.01095709,  0.        ,  0.60020816,  0.00491632,
         -1.1625783 ,  0.        ,  0.        , -0.3261763 ,
         -0.        ,  0.        ]],

       [[ 0.00356624,  0.        ,  0.        ,  0.00326727,
         -1.3008274 ,  0.        ,  0.06825623, -0.27123562,
         -0.        ,  0.        ]]], dtype=float32)>

y2 is equivalent to y1_drop/0.5.

When the dropout is applied to y1, with retention probability of p, then the output is then scaled by dividing it by p.

If you check both the matrices, y2 is nothing but dropping randomly half the inputs and then scaling it by 0.5.

# Long Short Term Memory

# Sequence Classification

In [ ]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
n_layers = 3

In [ ]:
learning_rate = 0.001

import tensorflow as tf
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [ ]:
lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
              for layer in range(n_layers)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
top_layer_h_state = states[-1][1] #[batch_size X num_neurons]
#top_layer_h_state is a tuple containing 3 LSTM layers, each with dimension batch_size X n_neurons
#this part is not doing softmax even though it is named as "softmax". 
logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")
#the shape of logits is batch_size X n_outputs
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

In [ ]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print("Epoch", epoch, "Train accuracy =", acc_train, "Test accuracy =", acc_test)

if you want to use peephole connections in LSTM, you can do:
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True)

If you want to use GRU cell, you can do:
gru_cell = tf.contrib.rnn.GRUCell(num_units=n_neurons)

# Binary String LSTM Example

Given a binary string (a string with just 0s and 1s) of length 20, we need to determine the count of 1s in a binary string. For example, “01010010011011100110” has 11 ones. So the input for our program will be a string of length twenty that contains 0s and 1s and the output must be a single number between 0 and 20 which represents the number of ones in the string http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/

In [ ]:
import numpy as np
from random import shuffle
 
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [ ]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [ ]:
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

#output is one-hot encoded

In [ ]:
n_inputs = train_input[0][0].shape[0] #1
n_outputs = len(train_output[0]) #21
n_steps = train_input[0].shape[0] #20
n_neurons = 24

In [ ]:
import tensorflow as tf
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

In [ ]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, state_is_tuple=True, forget_bias=1.0)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [ ]:
outputs= tf.transpose(outputs, [1, 0, 2])
last = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)

In [ ]:
learning_rate= 0.01

weights = {
    'out': tf.Variable(tf.truncated_normal([n_neurons, int(y.get_shape()[1])]))
}
biases = {
    'out': tf.Variable(tf.constant(0.1, shape=[y.get_shape()[1]]))
}

prediction = tf.matmul(last, weights['out'])+biases['out']
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#mistakes = tf.not_equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
#error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

batch_size = 1000
no_of_batches = int((len(train_input)) / batch_size)
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        ptr = 0
        for j in range(no_of_batches):
            inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
            ptr+=batch_size
            sess.run(opt,feed_dict={X: inp, y: out})
            if iter %10==0:
                acc=sess.run(accuracy,feed_dict={X: inp, y: out})
                los=sess.run(loss,feed_dict={X: inp, y: out})
                print("For iter ",iter)
                print("Accuracy ",acc)
                print("Loss ",los)
                print("__________________")
            iter=iter+1
        #calculating test accuracy
    print("Test Accuracy: ", sess.run(accuracy,{X:test_input, y:test_output }))

# Embeddings

This section is based on TensorFlow's Word2Vec tutorial
https://www.tensorflow.org/tutorials/representation/word2vec

In [ ]:
from six.moves import urllib

import errno
import os
import zipfile

WORDS_PATH = "datasets/words"
WORDS_URL = 'http://mattmahoney.net/dc/text8.zip'

def mkdir_p(path):
    """Create directories, ok if they already exist.
    
    This is for python 2 support. In python >=3.2, simply use:
    >>> os.makedirs(path, exist_ok=True)
    """
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def fetch_words_data(words_url=WORDS_URL, words_path=WORDS_PATH):
    os.makedirs(words_path, exist_ok=True)
    zip_path = os.path.join(words_path, "words.zip")
    if not os.path.exists(zip_path):
        urllib.request.urlretrieve(words_url, zip_path)
    with zipfile.ZipFile(zip_path) as f:
        data = f.read(f.namelist()[0])
    return data.decode("ascii").split()

In [ ]:
words = fetch_words_data()